In [4]:
%matplotlib widget

from sklearn.datasets import fetch_covtype
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import numpy as np

In [5]:
# load the full dataset and display it
dataset = fetch_covtype(shuffle=True) # make sure to shuffle the data so no local patterns emerge
feature_names = dataset.feature_names
target_names = dataset.target_names
data = dataset.data
target = dataset.target

Now knn algorithm is very good. It ignores many outliers if k is chosen to be appropriate and does a good job. However, we have another problem. Suppose we have 10 points. And also assume that we have 4 points for class 1 and 6 points for class 0. If we choose k to be 10, then knn algo will always choose class 0 as the category since there are more points compared to the other class. But what if class 1 was really close to our point we're trying to classify? No matter how close class 1 points get and how far class 0 points stray from, knn will still always pick class 0. This is a flaw in the knn algorithm. This happens because even though we use distance as a metric, once we choose k points, we completely ignore how the distances rank inside of those k points we picked. 

This motivates the need for a weighted knn algorithm. The idea is very similar to knn. We still pick k nearest points to our point, but instead of picking the most common category, we assign a weight to each of the points based on the distance it's from our point. Closer distances should rank higher and farther distances should rank lower. Therefore weight should be inversely proportional to the distance. A classic function is that of $w = 1/d$, where d is the distance.

In [6]:
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.33, random_state=1)
print("Train:", X_train.shape, y_train.shape)
print("Test:", X_test.shape, y_test.shape)

Train: (389278, 54) (389278,)
Test: (191734, 54) (191734,)


In [7]:
def predict_point(x, y, point, k):
    point = point.reshape((1,-1))
    x = x - point # mxn
    x2 = x**2 # squared euclidean distances
    dist_squared = np.sum(x2,axis=1) # add up all the squared euclidean distances

    k_indices = np.argsort(dist_squared)[:k] # get only the indices of the k minimum distances
    k_dist_squared = np.take(dist_squared, k_indices) # get the k minimum distances
    k_weights = 1 / k_dist_squared # compute 1/d weights
    k_categories = np.take(y, k_indices) # get the categories associated with the k indices

    weighted_sum = np.bincount(k_categories, weights = k_weights) # add the weights associated with the categories.
    category = np.argmax(weighted_sum) # pick the biggest weighted sum 

    return category # return the class associated with the k closest points

In [8]:
def predict_points(x, y, points, k):
    # same thing as predict_point but we handle multiple points now along with k points
    mp = points.shape[0]
    categories = np.zeros((mp))
    for i in range(mp):
        categories[i] = predict_point(x,y,points[i],k) # use predict_point we defined
    return categories

In [9]:
kvalues = np.round(np.linspace(1,50,5)).astype(np.int32)
print("K values to test:",kvalues)

K values to test: [ 1 13 26 38 50]


In [10]:
for i in range(kvalues.shape[0]):
    samples = 100
    predictions = predict_points(X_train, y_train, X_test[:samples], kvalues[i]) # try only 100 points from the testing set with k=100 points now
    accuracy = accuracy_score(predictions, y_test[:samples]) # compute the accuracy
    print(f"Accuracy for k={kvalues[i]} - {accuracy*100}%") # show the accuracy

Accuracy for k=1 - 91.0%
Accuracy for k=13 - 94.0%
Accuracy for k=26 - 93.0%
Accuracy for k=38 - 93.0%
Accuracy for k=50 - 93.0%


Now we see that for bigger k values, we get higher accuracies, and then eventually decreases again as k becomes too large. If you use a very noisy dataset or if the k value is too big, then this weighted knn should produce a much better result. It's important to note that accuracy for normal knn is higher than this one here. This is because the data is condensed and each separate category is spread out from each other. This clear separation of category and the patterns it exhibits make it fine to use normal knn or even simple nearest neighbor. However, we see here that the weighted version actually does do a lot better than the previous knn classifier for higher values of k. Around 9% accuracy increase for k=50. Therefore, choosing a good k becomes much less significant as you're ranking by the distance anyways. This makes it much better as you can avoid testing so many values of k and choose much less values to test instead. 

This lab has shown that each nearest neighbor classifier has its own pros and cons. You can't blindly pick any particular classifier without really understanding your data. For these labs, we've shown that many uncertain patterns emerge from our data and it turns out that simple nearest neighbor works really well our case. For other cases, you might have to choose knn or even weighted knn. It's important to be diligent with processes and check your data carefully before making an important decision. :)